# Simple Reflex Agent
Los simple reflex agents son programas que toman decisiones basadas únicamente en la información actual que reciben del entorno (percept), sin considerar el historial de percepciones pasadas. Funcionan siguiendo reglas de tipo "if-then" (si-entonces) para reaccionar de manera inmediata a los estímulos, pero no pueden planificar ni aprender de experiencias previas.

In [47]:
!python -m venv venv || source venvnb2\Scripts\activate
!pip install ollama random

Unable to copy 'C:\\Users\\Bo\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\venv\\scripts\\nt\\venvlauncher.exe' to 'c:\\gitprojects\\ses10\\venv\\Scripts\\python.exe'


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


En el ejemplo siguiente, se implementa un agente reflexivo simple que toma decisiones basadas únicamente en la percepción actual del entorno.

El agente recibe como entrada un diccionario con valores de temperatura y presión, y utiliza un conjunto de reglas de tipo "si-entonces" para determinar la acción a realizar (por ejemplo, encender o apagar la calefacción, activar o desactivar una bomba).

El agente evalúa cada regla y ejecuta la acción correspondiente si se cumple la condición, sin considerar el historial de percepciones previas ni mantener un modelo interno del entorno.

In [48]:
import random
import ollama

# Reglas de condición-acción
reglas = {
    "temperatura < 18": "encender calefacción",
    "temperatura > 24": "apagar calefacción",
    "presión < 30": "activar bomba",
    "presión > 70": "desactivar bomba"
}

In [49]:
# Función para evaluar condiciones simples
def cumple_condicion(percepcion, condicion):
    variable, operador, valor = condicion.split()
    valor = float(valor)
    entrada = percepcion.get(variable)

    if entrada is None:
        return False

    if operador == "<":
        return entrada < valor
    elif operador == ">":
        return entrada > valor
    elif operador == "==":
        return entrada == valor
    else:
        return False

# Simple Reflex Agent
def agente_reflejo_simple(percepcion):
    for condicion, accion in reglas.items():
        if cumple_condicion(percepcion, condicion):
            print(f"Acción local: {accion}")
            return accion

    print("Sin acción necesaria.")
    return "sin acción"


In [50]:
# Ejemplo de uso
percepcion_actual = {
    "temperatura": random.randint(10, 35), #15 in Celsius
    "presión": random.randint(20, 80) #45 in PSI
}
print(f"Percepción actual: {percepcion_actual}")
accion_tomada = agente_reflejo_simple(percepcion_actual)


Percepción actual: {'temperatura': 16, 'presión': 30}
Acción local: encender calefacción


# Model Based Reflex Agent
Un Model Based Reflex Agent es un agente que, además de reaccionar a los estímulos actuales (percepts), mantiene un modelo interno (model) del estado del entorno. Esto le permite tomar decisiones considerando no solo la percepción inmediata, sino también información sobre cómo el entorno puede cambiar con el tiempo. Así, puede manejar situaciones donde la percepción actual no es suficiente para decidir la acción correcta, usando su modelo para inferir el estado real del entorno.

In [51]:
!pip install langchain langchain_ollama langchain-openai python-dotenv

In [76]:
from langchain.chains.llm import LLMChain
from langchain_ollama.llms import OllamaLLM
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

# Configuración del LLM local con Ollama y modelo llama3.2
llm = OllamaLLM(model="llama3.2")

system_prompt = """Eres asistente inteligente de banca y finanzas.
Que decide la mejor acción en base a el estado actual de tu cliente.
Las acciones que puedes tomar son:
- mostrar saldo de cuenta ** esta no requiere confirmación del cliente **
- transferir dinero ** esta requiere confirmación del cliente **
- pagar factura ** esta requiere confirmación del cliente **
- invertir ** esta requiere confirmación del cliente **
** cuando estes a punto de realizar una acción, primero pregunta si el cliente desea proceder con la acción. De esa forma evitas acciones no deseadas y recoges el feedback del cliente. **
Tomas en cuenta tu historial de de tus acciones y confirmas si tu cliente decidio proceder o no con la accion que sugieras."""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])


In [77]:
pipeline = prompt_template | llm 

In [109]:
from langchain_core.chat_history import InMemoryChatMessageHistory
chat_map = {}

feedback_prompt = (
            """Devuelve un JSON estructurado con el feedback del cliente sobre la acción sugerida. 
            Incluye los siguientes campos: 'accion_sugerida', 'confirmacion_cliente' (True/False).
            Considera que la accion que estabas por tomar es: {action} y el feedback de cliente es: {userquery}"""
        )

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # Si no existe, crea un nuevo historial de chat en memoria
        chat_map[session_id] = InMemoryChatMessageHistory()
        return chat_map[session_id]
    print(history.messages)
    action = history.messages[-1].content if len(history.messages) > 0 else "ninguna"
    print (f"Accion tomada: {action}")
    userquery = history.messages[-2].content if len(history.messages) > 1 else "ninguno"
    print (f"Feedback del cliente: {userquery}")
    response = llm.invoke(feedback_prompt.replace("{action}", action).replace("{userquery}", userquery))
    chat_map[session_id].add_ai_message(str(response))
    return chat_map[session_id]

In [110]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [111]:
reply = pipeline_with_history.invoke(
    {"query": "hola, cual es mi saldo actual?"},
    config={"session_id": "id_123"}
)
print(reply)

Hola! Me alegra ayudarte con tu pregunta sobre el saldo actual de tu cuenta.

**Saldo Actual:** Se ha verificado tu saldo actual. Tu saldo actual es de $10,500.00.

¿Quieres realizar alguna otra acción o verificar algo más?


In [112]:
reply = pipeline_with_history.invoke(
    {"query": "Si, y si es muy alto, que puedo hacer?"},
    config={"session_id": "id_123"}
)
print(reply)

[]
Accion tomada: ninguna
Feedback del cliente: ninguno
**Sugerencia de Acción:** Considerando tu saldo actual de $10,500.00, podrías considerar invertir en una cartera diversificada para maximizar tus ganancias.

**Preguntando al cliente:**
Antes de proceder con la acción sugerida, ¿estás seguro de que deseas invertir un parte de tu saldo actual?


In [113]:
reply = pipeline_with_history.invoke(
    {"query": "Quiero invertir 5000 dolares en acciones tecnológicas."},
    config={"session_id": "id_123"}
)
print(reply)

[]
Accion tomada: ninguna
Feedback del cliente: ninguno
**Verificación de la Acción Sugerida**

Antes de proceder con la acción de invertir $5,000 en acciones tecnológicas, me aseguraré de verificar algunos requisitos importantes.

*   **Objetivo de Inversión**: ¿Cuál es tu objetivo de inversión? ¿Estás buscando ganancias a corto plazo o a largo plazo?
*   **Riesgo Tolerancia**: ¿Cuánto riesgo estás dispuesto a asumir con tus inversiones?
*   **Diversificación**: ¿Tienes una cartera diversificada de inversiones en diferentes activos, como acciones, bonos y bienes raíces?

**Recomendación**

Una vez que tenga más información sobre tus objetivos de inversión, tolerancia al riesgo y estrategia de diversificación, puedo ofrecerte recomendaciones personalizadas para tu caso específico.

Sin embargo, en este momento, puedo decirte que invertir $5,000 en acciones tecnológicas puede ser una opción a considerar, pero es importante tener en cuenta la volatilidad del mercado y los riesgos asociad

In [114]:
reply = pipeline_with_history.invoke(
    {"query": "Si, procede."},
    config={"session_id": "id_123"}
)
print(reply)

[]
Accion tomada: ninguna
Feedback del cliente: ninguno


KeyboardInterrupt: 

In [97]:

for msg in chat_map["id_123"].messages:
    print(f"{msg.type}: {msg.content}")

human: hola, cual es mi saldo actual?
ai: **Hola, ¡hoy te ayudaré a revisar tu saldo de cuenta!**

Antes de proceder, ¿te gustaría confirmar si deseas saber tu saldo actual? Si no necesitas esta información, puedo ofrecerte recomendaciones sobre cómo gestionar tus finanzas.
ai: Aquí tienes un ejemplo de cómo se podría estructurar este JSON:

```json
{
    "accion_sugerida": null,
    "confirmacion_cliente": false
}
```

Esto indica que la acción sugerida no fue tomada (nulo) y que el cliente confirmó que ninguna acción fue necesaria.
human: Si, y si es muy alto, que puedo hacer?
ai: **Revisando tu saldo actual...**

Tu saldo actual es de **$10,500.00**.

Antes de proceder a realizar cualquier acción, ¿te gustaría confirmar si deseas saber cómo gestionar tus finanzas o si hay algo específico en lo que te pueda ayudar?
ai: **Feedback del Cliente**
```json
{
  "accion_sugerida": "ninguna",
  "confirmacion_cliente": false
}
```

Esta respuesta indica que no se sugirió ninguna acción, pero 